#Building an AI assistant to book appointments using Google Calendar involves several steps, including setting up the API integration, creating a conversational agent with LangChain, and implementing conversation flow. Here’s a comprehensive guide to achieve this:

# Step 1: Set Up Google Calendar API Integration
Set Up Google Cloud Project:

Go to Google Cloud Console.
Create a new project or select an existing one.
Enable the Google Calendar API for your project:
Navigate to "APIs & Services" > "Library."
Search for "Google Calendar API" and enable it.

# 2. Create OAuth Credentials:

Go to "APIs & Services" > "Credentials."
Click "Create Credentials" and select "OAuth 2.0 Client IDs."
Configure the OAuth consent screen.
Create OAuth 2.0 Client IDs with the application type "Desktop App" or "Web Application" depending on your use case.
Download the credentials.json file.

# 3. Install Required Libraries:
   Install google-auth, google-auth-oauthlib, google-auth-httplib2, and google-api-python-client using pip:



In [5]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


   ---------------------------------------- 0.0/195.5 kB ? eta -:--:--
   -------------------- ------------------- 102.4/195.5 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 195.5/195.5 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.2/12.1 MB 6.1 MB/s eta 0:00:02
   - -------------------------------------- 0.5/12.1 MB 5.8 MB/s eta 0:00:03
   - -------------------------------------- 0.6/12.1 MB 5.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/12.1 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/12.1 MB 4.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.1 MB 4.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/12.1 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.1 MB 4.8 MB/s eta 0:00:03
   ------ --------------------------------- 2.0/12.1 MB 4.8 MB/s eta 0:00:03
   ---

In [ ]:
#4. Authorize and Access Google Calendar:

#Create a Python script to handle authentication and interact with Google Calendar:

from __future__ import print_function
import datetime
import os.path
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_google():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('calendar', 'v3', credentials=creds)

def create_event(service, summary, start_time, end_time):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'America/Los_Angeles',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'America/Los_Angeles',
        },
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    return event.get('htmlLink')

if __name__ == '__main__':
    service = authenticate_google()
    # Example usage
    event_link = create_event(service, 'Doctor Appointment', '2024-08-10T10:00:00-07:00', '2024-08-10T11:00:00-07:00')
    print(f'Event created: {event_link}')



In [ ]:
# Step 2: Build a Conversational Agent Using LangChain
# 1. Install LangChain:

pip install langchain


In [ ]:
# 2. Set Up LangChain Agent:
    # Create a Python script for the LangChain agent :
 from langchain import OpenAI, ConversationChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initialize OpenAI or other LLMs
openai_api_key = 'your-openai-api-key'  # Replace with your OpenAI API key
llm = OpenAI(api_key=openai_api_key)

# Define the conversation prompt
prompt_template = """
You are an AI assistant that books appointments for patients using Google Calendar. 
Your task is to understand the patient's request, such as date and time preferences, and book the appointment.

Patient's request: {user_input}

Provide a response indicating the booking status or ask for more details if needed.
"""

prompt = PromptTemplate(input_variables=['user_input'], template=prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)
conversation = ConversationChain(chain=chain)

def get_response(user_input):
    return conversation.run(user_input)

if __name__ == '__main__':
    user_input = "I need an appointment with my doctor next Monday at 10 AM."
    response = get_response(user_input)
    print(response)


In [ ]:
# Step 3: Implement Conversation Flow and Data Handling
# 1. Handle Appointment Requests:
     #Extend the conversation chain to parse user input, extract date and time, and call the Google Calendar API to create events.

import re
from dateutil.parser import parse

def extract_date_time(text):
    # Simple date/time extraction logic (expand as needed)
    date_match = re.search(r'(\d{4}-\d{2}-\d{2})', text)
    time_match = re.search(r'(\d{2}:\d{2}:\d{2})', text)
    if date_match and time_match:
        date_str = date_match.group(0)
        time_str = time_match.group(0)
        datetime_str = f"{date_str}T{time_str}"
        return parse(datetime_str)
    return None

def book_appointment(user_input):
    # Extract date and time from the input
    appointment_time = extract_date_time(user_input)
    if appointment_time:
        start_time = appointment_time.isoformat()
        end_time = (appointment_time + timedelta(hours=1)).isoformat()
        service = authenticate_google()
        event_link = create_event(service, 'Appointment', start_time, end_time)
        return f'Your appointment has been booked! You can view it here: {event_link}'
    else:
        return 'Sorry, I could not understand the date and time. Please provide them again.'

if __name__ == '__main__':
    user_input = "I need an appointment with my doctor on 2024-08-10 at 10:00:00."
    response = book_appointment(user_input)
    print(response)


In [ ]:
# Step 4: Test the Agent with Sample Conversations
# 1.Run Sample Tests:
#   Test with various user inputs to ensure the agent understands and correctly books appointments.
# 2.Refine Based on Feedback:
    #Improve date/time extraction and handling based on test results.

# By following these steps, we will have a functional AI assistant that uses Google Calendar to book appointments and handles user interactions naturally with LangChain. Be sure to handle errors and edge cases to improve the robustness of your assistant.